# Training the Detection Model

This notebook lets us train and fine-tune the Yolo-v5 model for the detection and classification for DeepFruitVision. All of the training and preparation is taken care of in other Python scripts, so this is just a centralized place to run the training and fine-tuning.

In [1]:
import os # we need this to change the working directory quite a lot
from modules.datasets import EnsembleDataset, save_dataset

from torch.utils.data import Subset
import numpy as np

Get the dataset ready.

In [2]:
os.chdir(os.path.join('..', 'modules'))

fine_tune_dir = os.path.join('..', 'dataset', 'yolov5_fine_tune')
fine_tune_val_dir = os.path.join(fine_tune_dir, 'val')
fine_tune_label_dir = os.path.join(fine_tune_dir, 'labels')
fine_tune_image_dir = os.path.join(fine_tune_dir, 'images')
fine_tune_val_image_dir = os.path.join(fine_tune_val_dir, 'images')
fine_tune_val_label_dir = os.path.join(fine_tune_val_dir, 'labels')

os.makedirs(fine_tune_label_dir, exist_ok=True)
os.makedirs(fine_tune_image_dir, exist_ok=True)

os.makedirs(fine_tune_val_label_dir, exist_ok=True)
os.makedirs(fine_tune_val_image_dir, exist_ok=True)

seed = 123
np.random.seed(seed)
num_fine_tune_samples = 50
experiment_index = 5 # YOU NEED TO CHANGE THIS BASED ON NUMBER OF 'EXPS' ARE IN THE TRAIN FOLDER

In [3]:
ensemble_dataset = EnsembleDataset(for_yolov5=True)

random_indices = np.random.permutation(len(ensemble_dataset))
fine_tune_indices = random_indices[:num_fine_tune_samples]
val_indices = random_indices[num_fine_tune_samples:]

fine_tune_dataset = Subset(ensemble_dataset, fine_tune_indices)
val_dataset = Subset(ensemble_dataset, val_indices)

Save the ensemble (used for fine tuning) and overall datasets to the disk for Yolo-v5 training

In [4]:
save_dataset(fine_tune_dataset, fine_tune_image_dir, fine_tune_label_dir)
save_dataset(val_dataset, fine_tune_val_image_dir, fine_tune_val_label_dir)

Saving dataset to ..\dataset\yolov5_fine_tune\images and ..\dataset\yolov5_fine_tune\labels: 100%|██████████| 50/50 [00:00<00:00, 145.10it/s]
Saving dataset to ..\dataset\yolov5_fine_tune\val\images and ..\dataset\yolov5_fine_tune\val\labels: 100%|██████████| 105/105 [00:00<00:00, 143.87it/s]


In [5]:
%run datasets.py

Splitting dataset: 100%|██████████| 6331/6331 [03:02<00:00, 34.73it/s]


apple: 4000 training objects, 1053 validation objects, 29895 testing objects
papaya: 1201 training objects, 305 validation objects, 383 testing objects
mango: 4000 training objects, 1005 validation objects, 17341 testing objects
487 empty images in training set, 132 empty images in validation set, 69 empty images in test set


Saving dataset to ..\dataset\images\train and ..\dataset\labels\train: 100%|██████████| 1432/1432 [00:13<00:00, 105.27it/s]
Saving dataset to ..\dataset\images\val and ..\dataset\labels\val: 100%|██████████| 371/371 [00:03<00:00, 108.94it/s]
Saving dataset to ..\dataset\images\test and ..\dataset\labels\test: 100%|██████████| 4528/4528 [00:58<00:00, 77.07it/s] 


Train the Yolo-v5 model

In [3]:
os.chdir(os.path.join('..', 'yolov5'))

In [7]:
%run train.py --data ../apple_papaya_mango.yaml --weights yolov5s.pt --img 416 --workers 4 --epochs 40 --hyp data/hyps/hyp.scratch-high.yaml

wandb: Currently logged in as: jvp15. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=../apple_papaya_mango.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=40, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github:  YOLOv5 is out of date by 1 commit. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_mom

Overriding model.yaml nc=80 with nc=3

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]             

metrics/mAP_0.5,▁▄▄▆▆▇▇▇▇▇▇█████████████████████████████
metrics/mAP_0.5:0.95,▁▃▃▅▄▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇████████████████
metrics/precision,▁▄▃▇▅▇▇▇▇▇▇▇█████▇██████████████████████
metrics/recall,▁▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇█▇██████▇████████
train/box_loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/cls_loss,█▇▅▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/obj_loss,█▇▅▅▅▃▄▃▄▄▃▃▃▃▂▂▃▂▃▃▂▂▂▂▂▂▂▂▂▂▃▂▁▁▁▂▂▁▁▁
val/box_loss,█▅▅▃▃▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/cls_loss,█▆▄▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/obj_loss,█▅▄▄▃▄▃▂▃▃▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
x/lr0,█▅▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Now let's evaluate the model on the test dataset (which is much larger than the validation dataset)

In [5]:
best_weights = os.path.join('runs', 'train', 'exp' + str(experiment_index), 'weights', 'best.pt')

In [9]:
%run val.py --data ../apple_papaya_mango.yaml --weights {best_weights} --img 416 --task test

val: data=../apple_papaya_mango.yaml, weights=['runs\\train\\exp5\\weights\\best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\labels\test... 4528 images, 69 backgrounds, 24 corrupt: 100%|██████████| 4528/4528 01:13
test: WARNING  C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\images\test\image_1279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0044]
test: WARNING  C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\ima

Now, we fine-tune Yolo-v5 on the fine-tune dataset

In [6]:
%run train.py --data ../fine_tune_apple_papaya_mango.yaml --weights {best_weights} --img 416 --workers 4 --epochs 40 --hyp data/hyps/hyp.scratch-high.yaml

wandb: Currently logged in as: jvp15. Use `wandb login --relogin` to force relogin
train: weights=runs\train\exp5\weights\best.pt, cfg=, data=../fine_tune_apple_papaya_mango.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=40, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github:  YOLOv5 is out of date by 4 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.000


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

metrics/mAP_0.5,▁▂▃▄▄▅▆▇▇▇▇▇▇▇▇▇▇▇▇█████████████████████
metrics/mAP_0.5:0.95,▁▂▂▄▄▅▆▆▇▇▇▇▇▇▇▆▇▇▆▇▇▇▇▇▇▇▇▇▇█▇▇████████
metrics/precision,▁▂▃▄▅▅▆▆▇▇▇▆▇▇█▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇█████
metrics/recall,▁▁▃▃▃▄▅▇▆▆▆▇▇▇▆▆▆▆▆▇▇█▇██▇▇████████▇████
train/box_loss,▆▆█▅▅▂▃▃▆▃▃▃▃▂▂▂▅▄▂▃▂▃▅▃▄▃▃▁▃▁▃▂▄▂▃▁▂▃▂▄
train/cls_loss,██▆▅▅▅▃▄▃▂▂▂▂▂▂▂▂▂▂▂▁▂▃▂▁▁▂▂▁▁▂▁▁▁▁▁▁▂▁▂
train/obj_loss,█▅▆▆▄▃▄▁▃▄▃▂▂▂▃▁▃▄▃▃▂▂▆▁▃▃▂▂▃▃▃▂▄▃▄▁▄▂▁▁
val/box_loss,▆▅▆▄▄▅▄▇▃▄▄▃▃▂▃█▅▃▇▃▅▄▆▆▇▅▄▄▃▂▃▂▂▂▂▁▁▁▁▁
val/cls_loss,█▇▅▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/obj_loss,█▇▇▇▆▇▇▇▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
x/lr0,██▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


And lastly, re-evaluate the model on the test dataset so that we know we didn't loose too much performance during fine-tuning

In [4]:
fine_tuned_weights = os.path.join('runs', 'train', 'exp' + str(experiment_index + 1), 'weights', 'best.pt')

In [5]:
%run val.py --data ../apple_papaya_mango.yaml --weights {fine_tuned_weights} --img 416 --task test

val: data=../apple_papaya_mango.yaml, weights=['runs\\train\\exp6\\weights\\best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs\val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5  2022-11-21 Python-3.10.5 torch-1.12.0 CUDA:0 (NVIDIA GeForce GTX 1070, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\labels\test.cache... 4528 images, 69 backgrounds, 24 corrupt: 100%|██████████| 4528/4528 00:00
test: WARNING  C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\dataset\images\test\image_1279.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0044]
test: WARNING  C:\Users\jorda\Documents\School\CMPE 295\fruit-detection\datas